In [1]:
# Membantu mem-fetch halaman
import requests
from requests.models import PreparedRequest
from selectolax.parser import HTMLParser
from random import random

# Berkaitan tentang waktu
from datetime import datetime, timedelta
from time import sleep

from typing import Optional  # pengingat tipe variabel, karena saya pikun
from tqdm import tqdm  # akan menampilkan progress-bar ketika looping

# Membantu mengolah data
import pandas as pd
from glob import glob
from ast import literal_eval

In [2]:
def get_data(filename: str):
  ''' Ekstrak data gempa dari filename'''

  with open(filename) as f:
    # baca berkas HTML
    text = f.read()

  # Ambil data gempa. Data terletak di tag <script> ketiga dari terakhir.
  the_script = HTMLParser(text).css('script')[-3].text()
  data = the_script.split('var locations =')[1].split(';\n')[0].strip()
    
  # Proses di atas akan mendapatkan data berupa list dalam format string.
  # Kita perlu mengubahnya ke format yang lebih manusiawi, yakni list.
  data = literal_eval(data) if data!='null' else []
    
  return data

In [3]:
total = []

# Semua berkas HTML yang sudah kita unduh berada di folder `raw`
for filename in tqdm(glob('./raw/*.html')):
    total.extend(get_data(filename))

# Ini ditulis dari masa depan. Semua data gempa yang bermasalah
# akan dicoba untuk didownload ulang; mereka akan diletakkan di
# folder `nasty`. 
for filename in tqdm(glob('./nasty/*.html')):
    total.extend(get_data(filename))

len(total)

100%|██████████| 26/26 [00:00<00:00, 533.51it/s]


102541

In [4]:
# Daftar kolom di data Gempa
columns = [
    'No', 'eventID', 'datetime', 'latitude', 'longitude', 'magnitude', 'mag_type',
    'depth', 'phasecount', 'azimuth_gap', 'location', 'agency',  'datetimeFM',
    'latFM', 'lonFM', 'magFM', 'magTypeFM', 'depthFM', 'phasecountFM', 'AzGapFM',
    'scalarMoment', 'Mrr', 'Mtt', 'Mpp', 'Mrt', 'Mrp', 'Mtp', 'varianceReduction',
    'doubleCouple', 'clvd', 'strikeNP1', 'dipNP1', 'rakeNP1', 'strikeNP2', 'dipNP2',
    'rakeNP2', 'azgapFM', 'misfit',
]

# buat DataFrame
df = pd.DataFrame(total, columns=columns)

# Beberapa perapian kecil:
# ubah kolom datetime dari string ke objek datetime
df.datetime = pd.to_datetime(df.datetime)
# buang kolom 'No' (Nomor)
df = df.drop(columns=['No'])
# buang data duplikat, jika ada
df = df.drop_duplicates()
# urutkan data berdasarkan datetime.
df = df.sort_values(by='datetime').reset_index(drop=True)

In [5]:
# cek selisih waktu event gempa dengan event sebelumnya
time_diff = df.datetime.sort_values().diff(periods=1)

# tampilkan semua selisih hari antar event gempa
set(time_diff.apply(lambda x: x.days))
# {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, nan, 27.0}

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, nan, 27.0}

In [6]:
df.tail(1)['datetime']

102514   2023-04-15 15:07:07.565140+00:00
Name: datetime, dtype: datetime64[ns, UTC]

In [7]:
df.to_csv('earthquakes.tsv', sep='\t')